In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

This is our master data file that has most of the data points thus far, and all remaining CSV's will be incorporated into this one:

In [3]:
df = pd.read_csv("CarvanaMasterData.csv")

In [4]:
df.head(2)

,Unnamed: 0,year_make,model,details,mileage,Bluetooth,Backup Camera,Infotainment,Screen,Navigation,...,4-Cyl,3-Cyl,V6,V8,6-Cyl,city_mpg,highway_mpg,No Reported Accidents,price,url
0,0,2016 Nissan,Altima,2.5 SR,"57,543 miles",1,1,1,0,0,...,1,0,0,0,0,26 City,37 Hwy,1,"$19,990",https://www.carvana.com/vehicle/2080019
1,1,2011 Honda,Civic,LX,"51,210 miles",0,0,1,0,0,...,1,0,0,0,0,25 City,36 Hwy,1,"$16,990",https://www.carvana.com/vehicle/2113974


First things first, drop any duplicates in the master CSV:

In [5]:
len(df[df.duplicated('url')])

1470

In [6]:
df.drop_duplicates('url', inplace=True)

In [7]:
len(df)

3722

In [8]:
df_list = []
file_names = ['Carvana2009Only.csv',
            'Carvana2010Only.csv',
            'Carvana2011Only.csv',
            'Carvana2012Only.csv',
            'Carvana2013Only.csv',
            'Carvana20092014.csv']

for file in file_names:
    df_by_year = pd.read_csv(file)
    df_list.append(df_by_year)

The data should have the same columns across all CSV's but we should set up a check to be sure

In [9]:
def check_columns(df1, df2):
    for i in range(len(df1.columns)):
        if df1.columns[i] != df2.columns[i]:
            print("Columns out of order at: " + str(df1.columns[i]) + ", " + df2.columns[i])
            return False

    if len(df1.columns) != len(df2.columns):
        print(f"Column count differs, {len(df1.columns)} vs {len(df2.columns)}")
        return False
    
    print("Columns equivalent")
    return True;

In [10]:
for df_other in df_list:
    check_columns(df, df_other)

Columns equivalent
Columns equivalent
Columns equivalent
Columns equivalent
Columns equivalent
Columns equivalent


In [11]:
df_copy = df.copy()

In [12]:
columns_added = 0
for df_other in df_list:
    columns_added += len(df_other)

In [13]:
# Total columns we can expect to be added to the master dataframe
columns_added

5714

In [14]:
# This is what the resulting length should be after concatentation
len(df) + columns_added

9436

In [15]:
# Concatenate the dataframes to the master df
for df1 in df_list:
    df_copy = pd.concat([df_copy, df1])

In [16]:
len(df_copy) == (len(df) + columns_added)

True

Success, we can now drop duplicates and clean up the data

In [17]:
df_copy.drop_duplicates('url', inplace=True)

In [18]:
len(df_copy)

7294

In [19]:
df_copy[df_copy.duplicated('url')]

,Unnamed: 0,year_make,model,details,mileage,Bluetooth,Backup Camera,Infotainment,Screen,Navigation,...,4-Cyl,3-Cyl,V6,V8,6-Cyl,city_mpg,highway_mpg,No Reported Accidents,price,url


In [20]:
df_copy.shape

(7294, 26)

In [21]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7294 entries, 0 to 1978
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             7294 non-null   int64 
 1   year_make              7294 non-null   object
 2   model                  7294 non-null   object
 3   details                7294 non-null   object
 4   mileage                7294 non-null   object
 5   Bluetooth              7294 non-null   int64 
 6   Backup Camera          7294 non-null   int64 
 7   Infotainment           7294 non-null   int64 
 8   Screen                 7294 non-null   int64 
 9   Navigation             7294 non-null   int64 
 10  Hands Free Calling     7294 non-null   int64 
 11  Heated Seats           7294 non-null   int64 
 12  Power Seat Controls    7294 non-null   int64 
 13  Rear Air Vents         7294 non-null   int64 
 14  Bed Liner              7294 non-null   int64 
 15  Tow Hitch            

In [22]:
df_copy.isnull().sum()

Unnamed: 0               0
year_make                0
model                    0
details                  0
mileage                  0
Bluetooth                0
Backup Camera            0
Infotainment             0
Screen                   0
Navigation               0
Hands Free Calling       0
Heated Seats             0
Power Seat Controls      0
Rear Air Vents           0
Bed Liner                0
Tow Hitch                0
4-Cyl                    0
3-Cyl                    0
V6                       0
V8                       0
6-Cyl                    0
city_mpg                 0
highway_mpg              0
No Reported Accidents    0
price                    0
url                      0
dtype: int64

Beautiful, there are a few things we need to clean up.

First one will be splitting "year_make" up into their own columns, "year" and "make"

In [23]:
df = df_copy.copy() # Turn it back into "df" to make typing it quicker everytime

In [24]:
df['year'] = df['year_make'].apply(lambda x: int(x.split(' ')[0]))
df['make'] = df['year_make'].apply(lambda x: x.split(' ')[1])

In [25]:
df.drop('year_make', inplace=True, axis=1)

In [26]:
df.head(2)

,Unnamed: 0,model,details,mileage,Bluetooth,Backup Camera,Infotainment,Screen,Navigation,Hands Free Calling,...,V6,V8,6-Cyl,city_mpg,highway_mpg,No Reported Accidents,price,url,year,make
0,0,Altima,2.5 SR,"57,543 miles",1,1,1,0,0,1,...,0,0,0,26 City,37 Hwy,1,"$19,990",https://www.carvana.com/vehicle/2080019,2016,Nissan
1,1,Civic,LX,"51,210 miles",0,0,1,0,0,0,...,0,0,0,25 City,36 Hwy,1,"$16,990",https://www.carvana.com/vehicle/2113974,2011,Honda


We can now clean up the mileage column to be an integer and remove all punctuation and text such as "miles"

In [27]:
df['mileage'] = df.mileage.apply(lambda x: x.replace(',', ''))
df['mileage'] = df.mileage.apply(lambda x: int(x.split(' ')[0]))

In [28]:
df.mileage.dtype

dtype('int64')

We can remove "City" and "Hwy" from city_mpg and highway_mpg

In [29]:
df['city_mpg'] = df.city_mpg.apply(lambda x: int(x.split(' ')[0]))
df['highway_mpg'] = df.highway_mpg.apply(lambda x: int(x.split(' ')[0]))


In [30]:
print(df.city_mpg.dtype)
print(df.highway_mpg.dtype)

int64
int64


In [31]:
df.head(3)

,Unnamed: 0,model,details,mileage,Bluetooth,Backup Camera,Infotainment,Screen,Navigation,Hands Free Calling,...,V6,V8,6-Cyl,city_mpg,highway_mpg,No Reported Accidents,price,url,year,make
0,0,Altima,2.5 SR,57543,1,1,1,0,0,1,...,0,0,0,26,37,1,"$19,990",https://www.carvana.com/vehicle/2080019,2016,Nissan
1,1,Civic,LX,51210,0,0,1,0,0,0,...,0,0,0,25,36,1,"$16,990",https://www.carvana.com/vehicle/2113974,2011,Honda
2,2,Focus,Titanium,19216,1,1,1,0,0,0,...,0,0,0,26,38,1,"$21,990",https://www.carvana.com/vehicle/2168176,2016,Ford


We can clean up the price column and cast it to an integer

In [32]:
price_chars = "$,"
for char in price_chars:
    df['price'] = df.price.apply(lambda x: x.replace(char, ''))

In [33]:
df['price'] = df.price.astype(int)

In [34]:
df.drop('No Reported Accidents', axis=1, inplace=True)

We can drop the 'No Reported accidents' feature, as it turns out that all cars on carvana have no reported accidents and it's unnecessary data

Finally, we can drop the URL, as it won't be necessary in the final data though it was very convenient for removing duplicates

In [35]:
df.drop('url', axis=1, inplace=True)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7294 entries, 0 to 1978
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           7294 non-null   int64 
 1   model                7294 non-null   object
 2   details              7294 non-null   object
 3   mileage              7294 non-null   int64 
 4   Bluetooth            7294 non-null   int64 
 5   Backup Camera        7294 non-null   int64 
 6   Infotainment         7294 non-null   int64 
 7   Screen               7294 non-null   int64 
 8   Navigation           7294 non-null   int64 
 9   Hands Free Calling   7294 non-null   int64 
 10  Heated Seats         7294 non-null   int64 
 11  Power Seat Controls  7294 non-null   int64 
 12  Rear Air Vents       7294 non-null   int64 
 13  Bed Liner            7294 non-null   int64 
 14  Tow Hitch            7294 non-null   int64 
 15  4-Cyl                7294 non-null   int64 
 16  3-Cyl 

In [37]:
df.head()

,Unnamed: 0,model,details,mileage,Bluetooth,Backup Camera,Infotainment,Screen,Navigation,Hands Free Calling,...,4-Cyl,3-Cyl,V6,V8,6-Cyl,city_mpg,highway_mpg,price,year,make
0,0,Altima,2.5 SR,57543,1,1,1,0,0,1,...,1,0,0,0,0,26,37,19990,2016,Nissan
1,1,Civic,LX,51210,0,0,1,0,0,0,...,1,0,0,0,0,25,36,16990,2011,Honda
2,2,Focus,Titanium,19216,1,1,1,0,0,0,...,1,0,0,0,0,26,38,21990,2016,Ford
3,3,Passat,1.8T Wolfsburg Edition,54167,1,0,1,0,0,1,...,1,0,0,0,0,24,34,17590,2014,Volkswagen
4,4,EcoSport,S,11168,1,1,1,0,0,0,...,1,0,0,0,0,23,29,23990,2021,Ford


In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.drop('index', axis=1, inplace=True)

Here I will export data for import into the next notebook which will perform EDA

In [ ]:
df.to_csv("CarvanaClean.csv")